In [1]:
#Importing libraries

import numpy as np
import pandas as pd
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
from scipy import stats
from scipy.stats import norm, skew
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

In [2]:
#Importing Data

train_df = pd.read_excel("C:/Users/kheni/Downloads/Train_set.xlsx")
test_df = pd.read_excel("C:/Users/kheni/Downloads/Test_set.xlsx")

In [3]:
#Combining Train and Test Data

big_df = train_df.append(test_df)

C:\Users\kheni\AppData\Local\Temp\ipykernel_12516\744722857.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_df = train_df.append(test_df)


In [4]:
#Printing out the data

big_df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0
...,...,...,...,...,...,...,...,...,...,...,...
2666,Air India,6/06/2019,Kolkata,Banglore,CCU → DEL → BLR,20:30,20:25 07 Jun,23h 55m,1 stop,No info,NaN
2667,IndiGo,27/03/2019,Kolkata,Banglore,CCU → BLR,14:20,16:55,2h 35m,non-stop,No info,NaN
2668,Jet Airways,6/03/2019,Delhi,Cochin,DEL → BOM → COK,21:50,04:25 07 Mar,6h 35m,1 stop,No info,NaN
2669,Air India,6/03/2019,Delhi,Cochin,DEL → BOM → COK,04:00,19:15,15h 15m,1 stop,No info,NaN


In [5]:
#Checking the data types of each column

big_df.dtypes

Airline             object
Date_of_Journey     object
Source              object
Destination         object
Route               object
Dep_Time            object
Arrival_Time        object
Duration            object
Total_Stops         object
Additional_Info     object
Price              float64
dtype: object

In [6]:
#Spliting Date_of_Journey column into three different columns i.e., Date, Month, and Year
#Converting the types to integer
#Dropping the old 'Date_of_Journey' column

big_df['Date'] = big_df['Date_of_Journey'].str.split('/').str[0]
big_df['Month'] = big_df['Date_of_Journey'].str.split('/').str[1]
big_df['Year'] = big_df['Date_of_Journey'].str.split('/').str[2]

big_df['Date'] = big_df['Date'].astype(int)
big_df['Month'] = big_df['Month'].astype(int)
big_df['Year'] = big_df['Year'].astype(int)

big_df=big_df.drop(['Date_of_Journey'], axis=1)

In [7]:
#Spliting the 'Arrival_Time' column

big_df['Arrival_Time'] = big_df['Arrival_Time'].str.split(' ').str[0]
big_df['Arrival_Time']

0       01:10
1       13:15
2       04:25
3       23:30
4       21:35
        ...  
2666    20:25
2667    16:55
2668    04:25
2669    19:15
2670    19:15
Name: Arrival_Time, Length: 13354, dtype: object

In [8]:
#Spliting Arrival_Time column into two different columns i.e., Hour, and Minutes

big_df['Arrival_Hour'] = big_df['Arrival_Time'].str.split(':').str[0]
big_df['Arrival_Minute'] = big_df['Arrival_Time'].str.split(':').str[1]

#Changing the data types of Arrival_Hours and Arrival_Minute

big_df['Arrival_Hour'] = big_df['Arrival_Hour'].astype(int)
big_df['Arrival_Minute'] = big_df['Arrival_Minute'].astype(int)

In [9]:
#Printing out 'Arrival_Hour' and 'Arrival_Minute'

big_df[['Arrival_Hour','Arrival_Minute']]

,Arrival_Hour,Arrival_Minute
0,1,10
1,13,15
2,4,25
3,23,30
4,21,35
...,...,...
2666,20,25
2667,16,55
2668,4,25
2669,19,15


In [10]:
#Dropping the old 'Arrival_Time' column

big_df = big_df.drop(['Arrival_Time'], axis = 1)

In [11]:
#Checking the number of NULL values in 'Total_Stops' column

big_df['Total_Stops'].isna().sum()

1

In [12]:
#Filling NULL value of 'Total_Stops' column to '1 Stop'

big_df['Total_Stops'] = big_df['Total_Stops'].fillna('1 Stop')

In [13]:
#Replacing 'non-stop' to '0 stop' in 'Total_Stops' column

big_df['Total_Stops'] = big_df['Total_Stops'].replace('non-stop', '0 stop')

In [14]:
#Spliting the 'Total_Stop' column and retriving the number of stops to 'Stop' column

big_df['Stop'] = big_df['Total_Stops'].str.split(' ').str[0]

In [15]:
#Printing out the 'Stop' column

big_df['Stop']

0       0
1       2
2       2
3       1
4       1
       ..
2666    1
2667    0
2668    1
2669    1
2670    1
Name: Stop, Length: 13354, dtype: object

In [16]:
#Dropping the old 'Total_Stops' column

big_df = big_df.drop(['Total_Stops'], axis = 1)

In [17]:
#Changing the datatype of 'Stop' column to integer

big_df['Stop'] = big_df['Stop'].astype(int)

In [18]:
#Printing out the 'Dep_Time' column

big_df['Dep_Time']

0       22:20
1       05:50
2       09:25
3       18:05
4       16:50
        ...  
2666    20:30
2667    14:20
2668    21:50
2669    04:00
2670    04:55
Name: Dep_Time, Length: 13354, dtype: object

In [19]:
#Spliting Dep_Time column into two different columns i.e., Hour, and Minute

big_df['Dep_Hour'] = big_df['Dep_Time'].str.split(':').str[0]
big_df['Dep_Minute'] = big_df['Dep_Time'].str.split(':').str[1]

In [20]:
#Printing out the 'Dep_Hour' and 'Dep_Minute' column

big_df[['Dep_Hour','Dep_Minute']]

,Dep_Hour,Dep_Minute
0,22,20
1,05,50
2,09,25
3,18,05
4,16,50
...,...,...
2666,20,30
2667,14,20
2668,21,50
2669,04,00


In [21]:
#Changing the datatype of 'Dep_Hour' and 'Dep_Minute' column to integer

big_df['Dep_Hour'] = big_df['Dep_Hour'].astype(int)
big_df['Dep_Minute'] = big_df['Dep_Minute'].astype(int)

In [22]:
#Dropping the old 'Dep_Time' column

big_df = big_df.drop(['Dep_Time'], axis = 1)

In [23]:
#Revising Route column by removing the arrow signs

big_df['Route_1'] = big_df['Route'].str.split('→').str[0]
big_df['Route_2'] = big_df['Route'].str.split('→').str[1]
big_df['Route_3'] = big_df['Route'].str.split('→').str[2]
big_df['Route_4'] = big_df['Route'].str.split('→').str[3]
big_df['Route_5'] = big_df['Route'].str.split('→').str[4]

In [24]:
#Checking for NULL values

big_df['Route_1'].isna().sum()

1

In [25]:
#Checking for NULL values

big_df['Route_2'].isna().sum()


1

In [26]:
#Checking for NULL values

big_df['Route_3'].isna().sum()


4341

In [27]:
#Checking for NULL values

big_df['Route_4'].isna().sum()

11397

In [28]:
#Checking for NULL values

big_df['Route_5'].isna().sum()

13296

In [29]:
#Replacing NaN with None in each Route columns to get rid of NULL values

big_df['Route_1'].fillna("None", inplace = True)
big_df['Route_2'].fillna("None", inplace = True)
big_df['Route_3'].fillna("None", inplace = True)
big_df['Route_4'].fillna("None", inplace = True)
big_df['Route_5'].fillna("None", inplace = True)

In [30]:
#Checking for NULL values in 'Price' column

big_df['Price'].isna().sum()

2671

In [31]:
#Imputing the mean value of price to all NULL values

big_df['Price'].fillna((big_df['Price'].mean()), inplace = True)

In [32]:
#Printing out the 'Price' column after imputing mean

big_df['Price']

0        3897.000000
1        7662.000000
2       13882.000000
3        6218.000000
4       13302.000000
            ...     
2666     9087.064121
2667     9087.064121
2668     9087.064121
2669     9087.064121
2670     9087.064121
Name: Price, Length: 13354, dtype: float64

In [33]:
#Used the decribe function to print out summary of statistics

big_df.describe()

,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Stop,Dep_Hour,Dep_Minute
count,13354.000000,13354.000000,13354.000000,13354.0,13354.000000,13354.000000,13354.000000,13354.000000,13354.000000
mean,9087.064121,13.389846,4.710574,2019.0,13.396061,24.664146,0.826045,12.513254,24.507264
std,4124.447805,8.439060,1.165622,0.0,6.896145,16.559723,0.674608,5.736273,18.832385
min,1759.000000,1.000000,3.000000,2019.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6135.250000,6.000000,3.000000,2019.0,8.000000,10.000000,0.000000,8.000000,5.000000
50%,9087.064121,12.000000,5.000000,2019.0,14.000000,25.000000,1.000000,11.000000,25.000000
75%,11087.000000,21.000000,6.000000,2019.0,19.000000,35.000000,1.000000,18.000000,40.000000
max,79512.000000,27.000000,6.000000,2019.0,23.000000,55.000000,4.000000,23.000000,55.000000


In [34]:
#Dropping out the old 'Route' and 'Duration' column

big_df = big_df.drop(['Route'], axis = 1)
big_df = big_df.drop(['Duration'], axis = 1)

In [35]:
#Preparing categorical variables for model using label encoder
#To convert categorical text data into model-understandable numerical data, we use the Label Encoder class

from sklearn.preprocessing import LabelEncoder

lb_encode = LabelEncoder()
big_df['Additional_Info'] = lb_encode.fit_transform(big_df['Additional_Info'])
big_df['Airline'] = lb_encode.fit_transform(big_df['Airline'])
big_df['Destination'] = lb_encode.fit_transform(big_df['Destination'])
big_df['Source'] = lb_encode.fit_transform(big_df['Source'])
big_df['Route_1'] = lb_encode.fit_transform(big_df['Route_1'])
big_df['Route_2'] = lb_encode.fit_transform(big_df['Route_2'])
big_df['Route_3'] = lb_encode.fit_transform(big_df['Route_3'])
big_df['Route_4'] = lb_encode.fit_transform(big_df['Route_4'])
big_df['Route_5'] = lb_encode.fit_transform(big_df['Route_5'])

In [36]:
#Finally checking the datatypes of each column

big_df.dtypes

Airline              int32
Source               int32
Destination          int32
Additional_Info      int32
Price              float64
Date                 int32
Month                int32
Year                 int32
Arrival_Hour         int32
Arrival_Minute       int32
Stop                 int32
Dep_Hour             int32
Dep_Minute           int32
Route_1              int32
Route_2              int32
Route_3              int32
Route_4              int32
Route_5              int32
dtype: object

In [37]:
#Dividing the data into train and test

df_train = big_df[0:10683]
df_test = big_df[10683:]

In [38]:
#Dropping the 'Price' column from the test dataset

df_test = df_test.drop(['Price'], axis = 1)

In [39]:
#Dividing df_train to X and y respectively

X = df_train.drop(['Price'], axis = 1)
y = df_train.Price

In [59]:
#Importing model libraries for Machine Learning models

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [68]:
#Assigning each to a variable

dt = DecisionTreeRegressor()
svr = SVR()
knn = KNeighborsRegressor()
lr = LinearRegression()

In [62]:
#Spliting the data by 70-30

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 47)

In [78]:
#Running for loop to fit and predict each model

for i in [dt, svr, knn, lr]:
    i.fit(X_train, y_train)
    pred = i.predict(X_test)
    test_score = r2_score(y_test, pred)
    train_score = r2_score(y_train, i.predict(X_train))
    #if abs(train_score - test_score) <= 0.1:
    print(i)
    print('R2 score is', r2_score(y_test, pred))
    print('R2 for train data is', r2_score(y_train, i.predict(X_train)))
    print('Mean Absolute Error is', mean_absolute_error(y_test, pred))
    print('Mean Squared Error is', mean_squared_error(y_test, pred))
    print('Root Mean Squared Error is', (mean_squared_error(y_test, pred, squared = False)))
    print('-----------------------------')

DecisionTreeRegressor()
R2 score is 0.8278857162847064
R2 for train data is 0.9965922757175513
Mean Absolute Error is 727.3556942277692
Mean Squared Error is 3703723.3102964116
Root Mean Squared Error is 1924.505991234221
-----------------------------
SVR()
R2 score is -0.021465086494998342
R2 for train data is -0.02507914415264345
Mean Absolute Error is 3649.1827690313457
Mean Squared Error is 21980883.68867493
Root Mean Squared Error is 4688.377511322539
-----------------------------
KNeighborsRegressor()
R2 score is 0.6570257833107811
R2 for train data is 0.7847906002964877
Mean Absolute Error is 1621.539282371295
Mean Squared Error is 7380454.2758564735
Root Mean Squared Error is 2716.6991507814173
-----------------------------
LinearRegression()
R2 score is 0.4978069836098046
R2 for train data is 0.49371557019830536
Mean Absolute Error is 2309.7338089909376
Mean Squared Error is 10806679.962420585
Root Mean Squared Error is 3287.3515118436276
-----------------------------


In [80]:
#Importing ensembling model libraries for Machine Learning model

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [82]:
#Assigning each to a variable

rfr = RandomForestRegressor()
abr = AdaBoostRegressor()
gbr = GradientBoostingRegressor()

In [83]:
#Spliting the data by 70-30

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [84]:
#Running for loop to fit and predict each model

for i in [rfr, abr, gbr]:
    i.fit(X_train, y_train)
    pred = i.predict(X_test)
    test_score = r2_score(y_test, pred)
    train_score = r2_score(y_train, i.predict(X_train))
    #if abs(train_score - test_score) <= 0.1:
    print(i)
    print('R2 score is', r2_score(y_test, pred))
    print('R2 for train data is', r2_score(y_train, i.predict(X_train)))
    print('Mean Absolute Error is', mean_absolute_error(y_test, pred))
    print('Mean Squared Error is', mean_squared_error(y_test, pred))
    print('Root Mean Squared Error is', (mean_squared_error(y_test, pred, squared = False)))
    print('-----------------------------')

RandomForestRegressor()
R2 score is 0.9041976904583602
R2 for train data is 0.9798230933912339
Mean Absolute Error is 605.3577255342594
Mean Squared Error is 1933315.9312879078
Root Mean Squared Error is 1390.4373165619181
-----------------------------
AdaBoostRegressor()
R2 score is 0.44576982861281733
R2 for train data is 0.5113897507794377
Mean Absolute Error is 2591.603152390914
Mean Squared Error is 11184511.365851225
Root Mean Squared Error is 3344.3252482154344
-----------------------------
GradientBoostingRegressor()
R2 score is 0.8323481943311677
R2 for train data is 0.8478499406895647
Mean Absolute Error is 1205.400261375095
Mean Squared Error is 3383257.756096785
Root Mean Squared Error is 1839.3634105572462
-----------------------------


In [86]:
# Performing Cross Validation to remove overfitting, if any
#Importing cross_val_score library for Cross Validation

from sklearn.model_selection import cross_val_score

In [89]:
#Running for loop for CV from 2 to 9
#This is done on Random Forest Regressor

for i in range(2,9):
    cv = cross_val_score(rfr, X, y, cv = i)
    print(rfr, cv.mean())

RandomForestRegressor() 0.8757826175497462
RandomForestRegressor() 0.8797052920201892
RandomForestRegressor() 0.8925452156716971
RandomForestRegressor() 0.8957933798246488
RandomForestRegressor() 0.8963873764899061
RandomForestRegressor() 0.9003574686331718
RandomForestRegressor() 0.8980018954623677


In [90]:
#Running for loop for CV from 2 to 9
#This is done on Gradient Boosting Regressor

for i in range(2,9):
    cv = cross_val_score(gbr, X, y, cv = i)
    print(gbr, cv.mean())

GradientBoostingRegressor() 0.8235574382031221
GradientBoostingRegressor() 0.8260138362588064
GradientBoostingRegressor() 0.8278407648418287
GradientBoostingRegressor() 0.8295598539688864
GradientBoostingRegressor() 0.8274151906532365
GradientBoostingRegressor() 0.8295296751639843
GradientBoostingRegressor() 0.8296026240744189


In [91]:
#Hypertuning the model using GridSearchCV

from sklearn.model_selection import GridSearchCV

In [98]:
#GridSearchCV for Random Forest Regressor

param_grid = {'n_estimators': [10,30,50,70,100], 'max_depth': [None, 1, 2, 3], 'max_samples': [50,100,250,500,1000], 
              'min_samples_split': [2,4,10]}

gscv_rfr = GridSearchCV(rfr, param_grid, cv = 3)

In [101]:
# Fitting the model

res = gscv_rfr.fit(X_train, y_train)

In [103]:
# Checking out the best parameter

res.best_params_

{'max_depth': None,
 'max_samples': 1000,
 'min_samples_split': 2,
 'n_estimators': 100}

In [120]:
#Printing out the result achieved by the best parameter

res.best_score_

0.8150666888236513

In [113]:
#GridSearchCV for Gradient Boosting Regressor

param_grid_2 = {'alpha': [0.9,0.09,0.1], 'learning_rate':[0.1,0.01], 'max_depth': [3,4,5], 'min_samples_leaf': [1,2,3],
               'min_samples_split': [2,3,4], 'n_estimators': [100,50,10]}

In [114]:
gscv_gbr = GridSearchCV(gbr, param_grid_2, cv=3)

In [116]:
#Fitiing the model

res2 = gscv_gbr.fit(X_train, y_train)

In [117]:
#Checking out the best parameter

res2.best_params_

{'alpha': 0.9,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 100}

In [119]:
#Printing out the result achieved by the best parameter

res2.best_score_

0.8456124050012215

In [123]:
#We got our best model with a score of 84.5% (Gradient Boosting Regressor)
#Saving it using best parameter and creating a model object using joblib

model = GradientBoostingRegressor(alpha = 0.9, learning_rate = 0.1, max_depth = 5, min_samples_leaf = 1, min_samples_split = 4,
                                 n_estimators = 100)


In [148]:
#Importing joblib library and storing our model in object named flight_price.obj

import joblib

joblib.dump(model, 'flight_price.obj')

['flight_price.obj']

In [136]:
#Predicting X_test on our best model and printing out the results side by side

model = joblib.load('flight_price.obj')
model.fit(X_train, y_train)
pred = model.predict(X_test)

predicted_values = pd.DataFrame({'Actual': y_test, 'Predicted': pred})
predicted_values

,Actual,Predicted
6041,3597.0,3804.404450
5637,3383.0,4020.716200
9644,2050.0,3231.028773
3159,4423.0,4207.168513
5278,3597.0,4061.426159
...,...,...
6821,13731.0,13882.662538
10518,9416.0,10618.466659
397,3597.0,3783.702023
9847,5277.0,5238.228357


In [152]:
#Predicting our best model on the new data (test data)

flight_price = joblib.load('flight_price.obj')

prices = flight_price.predict(df_test)

In [158]:
#Printing out the predicted values

prices

array([14059.536814  ,  5470.02920794, 11532.88225686, ...,
       16923.35919605, 14158.48020774,  9226.63359671])

In [160]:
#Converting the predicted values (array) into a dataframe

price_list = pd.DataFrame({'Price': prices})
price_list

,Price
0,14059.536814
1,5470.029208
2,11532.882257
3,10684.572294
4,3736.203885
...,...
2666,9405.328788
2667,4820.390114
2668,16923.359196
2669,14158.480208
